This is a eMail Spam Classifers that uses Naive Bayes supervised machine learning algorithm.

In [1]:
#import needed packages into colab
import os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from google.colab import drive

#Mounted the google drive to make files readable.
drive.mount('/content/drive/')

Mounted at /content/drive/


This function builds a Dictionary of most common 3000 words from all the email content. First it adds all words and symbols in the dictionary. Then it removes all non-alpha-numeric characters and any single character alpha-numeric characters. After this is complete it shrinks the Dictionary by keeping only most common 3000 words in the dictionary. It returns the Dictionary.

In [2]:
# Data cleaning
# Remove the non required words, expressions and symbols from text.

def make_Dictionary(root_dir):                # define a new function
  all_words_list = []                         # create a empty list where we can put most common words in it.
  allemails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)]   # create a list called allemails
                                              # concatenates various path components with exactly one directory separator into a string
                                              # and also retrieves the list with all the files in it
  for mail in allemails:                      # read single item in 'allemails' list; define mail inside the allemails
    with open(mail) as m:                     # after find mail, we will open each of the mail, and give it a name 'm'
      for line in m:                          # loop single item in single file 'm'
        words = line.split()                  # split each line (string) and convert them into a list
        all_words_list += words               # add all words into the empty dictionary list
  dictionary_lookup = Counter(all_words_list) # create a new container called dictionary_lookup, which contains all of the words we found for the dictionary, and will list how many times equivalent values are added
  list_to_remove = list(dictionary_lookup)    # concert the 'dictionary_lookup' into a list, so that we can search for common words later.
                                         

  for singleitem in list_to_remove:                        #iterate the list
    if singleitem.isalpha() == False:           
      del dictionary_lookup[singleitem]                    #remove all the non-alpha numeric characters
    elif len(singleitem) == 1:                  
      del dictionary_lookup[singleitem]                    #remove any single character alpha-numeric characters 
  dictionary_lookup = dictionary_lookup.most_common(3000)  # only keep the most common 3000 words in the dictionary_lookup list  
  return dictionary_lookup                                 # return the dictionary_lookup

  # In concusion, after this part of data collection and data cleaning, we can get a 3000 most common words from the train dataset.
  # Spam emails have already becoming a huge problem on the internet, with Machine Learning, we can easily create a dictionary 
  # which contain most of the common words when we consider a spam email.

This function extracts feature columns and populates their values (Feature Matrix of 3000 comumns and rows equal to the number of email files). The function also analyzes the File Names of each email file and decides if it's a Spam or not based on the naming convention. Based on this the function also creates the Labelled Data Column. This function is used to extract the training dataset as well as the testing dataset and returns the Feature Dataset and the Label column.

In [3]:
# extract features

def extract_features(mail_dir):                               #Define a new function to extract the data
  files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)] # create a list called files
                                                              # concatenates various path components with exactly one directory separator into a string
                                                              # and also retrieves the list with all the files in it
  features_matrix = np.zeros((len(files),3000))               # Also use np.zero function to return a new array with 3000 columns and the same number of email files as rows with zeros.(template empty)
  train_labels = np.zeros(len(files))                         # return another array contains the number of email files as row numbers.
  count = 1;                                                  # start of counter set to 1
  docID = 0;                                                  # start of docID set to 0
  for file in files:                                          # loop each file from 'files' list
    with open(file) as f:                                     # open each single file as f
      for itemcount, item in enumerate(f):                    # itemcount is the count of items, item is the item itself. Use enumerate method adds counter to an iterable and returns it, the returned object is a enumerate object
        if itemcount == 2:                                    # if the count is 2
          words = item.split()                                # we will split the line in to a list called words
          for word in words:                                  # read each single item in list 'words'
            wordID = 0                                        # set wordID to 0
            for itemcount, item2 in enumerate(dictionary_lookup):   
                                                              # loop dictionary_lookup list. itemcount is the count of items, and item2 is the item in dictionary_lookup
              if item2[0] == word:                            # if the first index/item in the list equal to word (in words)
                wordID = itemcount                            # then we assign itemcount to wordID 
                features_matrix[docID,wordID] = words.count(word)    #count how many unique word in words   [1,1,2,3]   2：1， 1：2， 1：3
      train_labels[docID] = 0;                                       # set train_labels as 0
      filepathTokens = file.split('/')                               # split each character backslash
      lastToken = filepathTokens[len(filepathTokens)-1]              # since python count from 0, we have to minus one as it is last word
      if lastToken.startswith("spmsg"):                       # according to the case instruction, when an title of an email begin with 'spmsg' is means file is of spam emails.
                                                              # The function also analyzes the File Names of each email file and decides if it's a Spam or not based on the naming convention. 
                                                              # Based on this the function also creates the Labelled Data Column
        train_labels[docID] = 1;                              # set train_label as 1
        count = count + 1                                     # the number of count will be added one by each loop
      docID = docID + 1                                       # the number of count of docID will be added one by each loop
  return features_matrix, train_labels                        #turn the results

  #In summary, after this part of coding, we will get a new function
  #This function that can help us to extract the training dataset and the testing dataset 
  #in order to return the Feature Dataset and the Label column.

The section is the main Program that calls the above two functions and gets executed first. First it "trains" the model using model.fit function and Training Dataset. After that it scores the Test Data set by running the Trained Model with the Test Data set. At the end it prints the model performance in terms of accuracy score.

In [4]:
#import Train & Test data

TRAIN_DIR = '/content/drive/My Drive/MSBA_Colab_2020/ML Algorithms/Naive_Bayes/train-mails'
TEST_DIR = '/content/drive/My Drive/MSBA_Colab_2020/ML Algorithms/Naive_Bayes/test-mails'

In [5]:
#Gaussian model is apply for the classification and it assumes that features follow a normal distribution.
#call two functions

dictionary_lookup = make_Dictionary(TRAIN_DIR)

print ("reading and processing emails from TRAIN and TEST folders")
features_matrix, labels = extract_features(TRAIN_DIR)           
test_features_matrix, test_labels = extract_features(TEST_DIR)  

model = GaussianNB()

print ("Training Model using Gaussian Naibe Bayes algorithm .....")
model.fit(features_matrix, labels)                           #use train data to train the model
print ("Training completed")
print ("testing trained model to predict Test Data labels")
predicted_labels = model.predict(test_features_matrix)       #use test data to perdict
print ("Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")
print (accuracy_score(test_labels, predicted_labels))        

# Modle scores the test data by running the trained model with the test data.
# Accurancy score is the percentage of correct perdiction, which is around 0.96 after we run the model. Also means this model is accurate for perdiction.

reading and processing emails from TRAIN and TEST folders
Training Model using Gaussian Naibe Bayes algorithm .....
Training completed
testing trained model to predict Test Data labels
Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:
0.9653846153846154


======================= END OF PROGRAM =========================